# Problem Set 4: Parabolic Problems

In [1]:
import numpy as np
import scipy as sp
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve
from scipy.io import loadmat
import matplotlib.tri as tri
import matplotlib.pyplot as plt
from time import process_time_ns
import random
from time import process_time_ns

Our problem of interest is the thermal fin discussed in the previous problem sets, but now we consider the time-dependent case. We assume that the thermal fin is initially at zero (non-dimensionalized) temperature and a heat flux is then applied to the root. The output of interest is the average temperature of the fin. We directly consider the truth approximation. To this end, we divide the time interval, $I = [0, t_f ]$, into K subintervals of equal length $\Delta = \frac {t_f}{K}$, and define $t_k = k∆t, 0 ≤ k ≤ K$. 

We shall consider Euler-Backward for the time integration. We also recall the truth finite element approximation space $X ⊂ X^e$ .

Our truth problem statement is then: given a parameter $μ ∈ D$, we evaluate the output
$$s^k (μ) = l(u^k (μ)), 1 ≤ k ≤ K,$$
where the field variable $u^k (μ) ∈ X, 1 ≤ k ≤ K,$ satisfies:
$$ m\big (\frac {u^k (μ) − u^{k−1} (μ)}{\Delta t}, v \big ) + a(u^k (μ), v; μ) = f (v)g(t^k ), \quad ∀v ∈ X$$

with initial condition $u(t_0 ; μ) = u_0 =  0$. Here, the bilinear form a is defined as in Problem Set 1,
the linear form f is given by $f (v) = \int_{Γ_{root}} v$, the linear form l is given by $l(v) = \int_Ω v$, the bilinear form m is given by:
$$m(u,v) = \int_Ω uv, \forall u,v \in X$$
and $g(t_k )$ denotes the “control input” at time $t = t_k$. Note that m and l, f are parameter-independent.

We consider the following special case: We assume that the conductivities of all fins are equivalent
and fixed at $k_i = 1, i = 1, ..., 4$, and that the Biot number is allowed to vary between 0.01 and 1. We
thus have $μ ≡ Bi ∈ D = [0.01, 1]$. We consider the time interval $I = [0, 10]$ with a discrete timestep
$∆t = 0.1$ and thus $K = 100$.

# Part 1 - Reduced Basis Approximation

We first generate a reduced basis approximation by choosing a basis from scratch. To this end, we use $g(t_k ) = δ_{1 k} , 1 ≤ k ≤ 100$ (unit impulse input) and set:
$$X_N = span \{u^1 (0.01), u^5 (0.01), u^{10} (0.01), u^{20}(0.01), u^{30}(0.01), u^5 (0.1), u^{10}(0.1), u^{20} (0.1), u^5 (1), u^{10} (1) \}$$

i.e., our reduced basis space X N is spanned by the solution $u^k(μ)$ at several parameter-time pairs.
We then orthonormalize $X_N$ using Gram-Schmidt.


In [26]:
class RB_PB4:
    def __init__(self,N=10,Q=6,triangulation ='medium',dt=0.1,K=100):
        self.N = N # RB dimension
        self.Q = Q #number of parameters
        self.triangulation = triangulation #'coarse', 'medium' or 'fine'
        self.dt = dt # time step
        self.K = K #number of subintervals
        
        self.N_cal = None #dimention of FE space
        self.Aq_mesh = None # FE matrix stiffness independant of parameters
        self.M_mesh = None # FE matrix_mass independant of parameters
        self.Fh_mesh = None # FE second term
        self.L_mesh = None # metric or the output vector
        self.G = np.zeros((self.K,1))
        self.G[1] = 1
        
        self.mu_bar=np.array([1,1,1,1,1, 0.1])
        self.A_mu_bar = None #X inner product associated matrix
        
        '''Load matrices, mus, mu hat and samples'''
        self.load_matrices_samples()
        
        
    def load_matrices_samples(self):
        '''stiffness matrix'''
        A = loadmat('FE_matrix.mat',simplify_cells=True)
        self.Aq_mesh=A['FE_matrix'][self.triangulation]['Ahq']
        self.Fh_mesh=A['FE_matrix'][self.triangulation]['Fh']
        
        '''mass matrix'''
        M = loadmat('FE_matrix_mass.mat',simplify_cells=True)
        self.M_mesh=M['FE_matrix_mass'][self.triangulation]['Mh']
        #print(self.Mq_mesh.toarray().shape)

        '''grid'''
        grids = loadmat('FE_grid.mat',simplify_cells=True)
        self.N_cal = grids['FE_grid'][self.triangulation]['nodes']
        
        '''X inner product associated matrix'''
        self.A_mu_bar = self.assembleA(self.mu_bar,self.Aq_mesh)
        
        '''the output vector'''
        self.L_mesh = self.M_mesh@np.ones(self.N_cal)
        #print(self.L_mesh)
        
        '''Orthonomalzed RB'''
        self.Z_ort = np.zeros((self.N_cal,self.N))
        
    def inner_prod_A(self,u,v):
        return u.T@self.A_mu_bar@v
    
    def assembleA(self,mu,Aq):
        A=csc_matrix(np.shape(Aq[0]))
        for k in range(0,6):
            A+=mu[k]*Aq[k]
        return A
    
    def Gram_Schmidt_inplace(self,u,i):
        v = u.copy()
        
        for k in range(i):
            v[:,i]-=self.inner_prod_A(u[:,i],u[:,k])*u[:,k]
        v[:,i] = v[:,i]/np.sqrt(self.inner_prod_A(v[:,i],v[:,i]))
        return v[:,i]

    
    def single_offline_calculation_RB(self,Bi,k,c,ki=1):        
        mu=np.array([ki,ki,ki,ki,1,Bi])
        A = self.A_mu_bar
        M = self.M_mesh
        F = self.Fh_mesh
        G = self.G
        dt = self.dt
        
        u= np.zeros(self.N_cal) #initialization
        for i in range(1,k):
            u = spsolve(M/dt+A, F*G[i] + M@u/dt)#a la place de la decomp LU
        self.Z_ort[:,c] = u[:]
        
    def orthonormalize_RB(self):
        for c in range(self.N):
            self.Z_ort[:,c] = self.Gram_Schmidt_inplace(self.Z_ort,c)
        
    def offline_calculation_RB(self):
        
        BI = [0.01]*5 + [0.1]*3 + [1]*2
        K = [1,5,10,20,30,5,10,20,5,10]
        c=0
        for Bi,k in zip(BI,K):
            self.single_offline_calculation_RB(Bi,k,c)
            c+=1
            
    def G_func(self):
        t = np.arange(0,K*self.dt,self.dt)
        out = 1 - np.cos(t)
        
    def calculate_outputs(self,Bi):
        mu = [1,1,1,1,1,Bi] 

        G = self.G_func()
        dt = self.dt
        
        '''finit element'''
        M = self.M_mesh
        F = self.Fh_mesh
        A = self.assembleA(mu,self.Aq_mesh)
        
        self.S = []
        for i in range(1,self.K):
            u = spsolve(M/dt+A, F*G[i] + M@u/dt)#a la place de la decomp LU
            self.S.append(self.L_mesh.T@u)
            
        '''RB'''
        AN = self.Z_ort@A@self.Z_ort
        MN = self.Z_ort@self.M_mesh@self.Z_ort
        FN = self.Z_ort@self.Fh_mesh
        LN = self.Z_ort@self.L_mesh;
        
        self.SN = []
        for i in range(1,self.K):
            uN = spsolve(MN/dt+AN, FN*G[i] + MN@uN/dt)#a la place de la decomp LU
            self.SN.append(self.L_mesh.T@uN)
        
        

In [27]:
def test_PB4():
    test = RB_PB4(N=10,Q=6,triangulation ='medium')
    test.offline_calculation_RB()
    print(test.Z_ort)
    test.orthonormalize_RB()
    print(test.Z_ort)

test_PB4()

[[0.         0.08644676 0.0435844  ... 0.01955536 0.08644676 0.0435844 ]
 [0.         0.08644233 0.04358541 ... 0.0195561  0.08644233 0.04358541]
 [0.         0.03013768 0.02721055 ... 0.01522139 0.03013768 0.02721055]
 ...
 [0.         0.0816752  0.04319629 ... 0.01969208 0.0816752  0.04319629]
 [0.         0.08129207 0.04296297 ... 0.01958554 0.08129207 0.04296297]
 [0.         0.08115907 0.04293544 ... 0.01957949 0.08115907 0.04293544]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


/home/you/.local/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide


## Question 1:

The following formula was used to build the RB in the class above:

$$ m\big (\frac {u^k (μ) (μ)}{\Delta t}, v \big ) + a(u^k (μ), v; μ) = f (v)g(t^k )+ m\big (\frac {u^{k−1} (μ)}{\Delta t}, v \big ), \quad ∀v ∈ X$$


I did not succeed to find the orthonormalization formula! so the following results are obtained with a non-orthonormalized RB.

# Part 2 - A Posterior Error Estimation

## Question 2: